In [109]:
# import necessary packages
import numpy as np
import pandas as pd

In [110]:
# import dataset
df = pd.read_csv('https://raw.githubusercontent.com/notpeter/crunchbase-data/master/companies.csv')
# df.head()


Cleaning data import values for nulls, errors and column types

In [111]:
df['funding_total_usd'] = df['funding_total_usd'].replace('-', np.nan)
df['funding_total_usd'].fillna(0, inplace=True)
df['funding_total_usd'] = df['funding_total_usd'].astype(float)
df.loc[:, 'founded_at'] = df['first_funding_at']
df.dropna(subset=['name'], inplace=True)
# Remove comment to view null data from a specific column
# df[df['country_code'].isnull()]
# Remove comment to review total errors per column
# df.isnull().sum()

### Measuring Central Tendency and Spread

Calculate measures of central tendency and spread for a startup

In [112]:
# Calculate measures of central tendency for total funding raised
mean_total_funding = df['funding_total_usd'].mean()
median_total_funding = df['funding_total_usd'].median()
mode_total_funding = df['funding_total_usd'].mode().values[0]

# Calculate measures of spread for total funding raised
range_total_funding = df['funding_total_usd'].max() - df['funding_total_usd'].min()
variance_total_funding = df['funding_total_usd'].var()
std_deviation_total_funding = df['funding_total_usd'].std()

# Calculate measures of central tendency for number of funding rounds
mean_funding_rounds = df['funding_rounds'].mean()
median_funding_rounds = df['funding_rounds'].median()
mode_funding_rounds = df['funding_rounds'].mode().values[0]

# Calculate measures of spread for number of funding rounds
range_funding_rounds = df['funding_rounds'].max() - df['funding_rounds'].min()
variance_funding_rounds = df['funding_rounds'].var()
std_deviation_funding_rounds = df['funding_rounds'].std()

# Print the results
print("Total Funding Raised:")
print(f"Mean: {mean_total_funding:.2f}")
print(f"Median: {median_total_funding:.2f}")
print(f"Mode: {mode_total_funding:.2f}")
print(f"Range: {range_total_funding}")
print(f"Variance: {variance_total_funding:.2f}")
print(f"Standard Deviation: {std_deviation_total_funding:.2f}")

print("\nNumber of Funding Rounds:")
print(f"Mean: {mean_funding_rounds:.2f}")
print(f"Median: {median_funding_rounds:.2f}")
print(f"Mode: {mode_funding_rounds:.2f}")
print(f"Range: {range_funding_rounds:.2f}")
print(f"Variance: {variance_funding_rounds:.2f}")
print(f"Standard Deviation: {std_deviation_funding_rounds:.2f}")

Total Funding Raised:
Mean: 14919146.79
Median: 1000000.00
Mode: 0.00
Range: 30079503000.0
Variance: 28592863519409792.00
Standard Deviation: 169094244.49

Number of Funding Rounds:
Mean: 1.73
Median: 1.00
Mode: 1.00
Range: 18.00
Variance: 1.85
Standard Deviation: 1.36


For "Total Funding Raised" the average funding raised by startups is $14,919,146 USD with an average of 1.7 rounds of funding per startup but it is highly likely that some startups have received exceptionally large funding amounts, which significantly skews the distribution. The mean is much higher than the median, indicating the presence of high-value outliers because probably few startups have received exceptionally large amounts of funding. The mode proves this too because there are many startups with 0 funding (but can also be due to missing data). The variance and range are very large which indicates a presence of outliers at the upper end of the distribution.

There is a lot less variation in the "Funding rounds" and when observing the difference of measures of central tendency which suggests that the data may be less affected by outliers. The measure of spread also contain moderate levels of variance which indicate that the outliers are not as noticeable.


### Outlier analysis